In [1]:
import os
import time
import json
import numpy as np
import pandas as pd
import wandb
import math
import random
from typing import Tuple, Sequence, Callable
from PIL import Image

import torch
import torch.optim as optim
import torch.nn.functional as F
from torch import nn, Tensor
from torch.utils.data import Dataset, DataLoader
from torchinfo import summary
from torchvision import transforms
from torchvision.models import *
from sklearn.preprocessing import LabelEncoder

from material import MyDataset

In [2]:
torch.manual_seed(12345)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(12345)
random.seed(12345)

In [3]:
#gpu 사용을 위해
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [4]:
def transform(proba):
    t = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(p=proba),
        transforms.RandomVerticalFlip(p=proba),
        transforms.ToTensor(),
        transforms.Normalize(
            [0.485, 0.456, 0.406],
            [0.229, 0.224, 0.225]
        )
    ])
    return t

In [5]:
trainsets = []
dirs = ["leather/", "mesh_knit/", "suede/", "nylon/"]
prob = [0.25, 0.5, 1, 0.75]

for i in range(4):
    dataset = MyDataset(dir='/home/compu/Documents/exports/material_merge/train/'+dirs[i],
                        image_ids='/home/compu/Documents/exports/newfile_material_merge.json',
                        transforms=transform(prob[i]))
    trainsets.append(dataset)
    print(len(dataset))

trainset = torch.utils.data.ConcatDataset(trainsets)

11129
5918
2063
4087


In [6]:
transforms_test = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.485, 0.456, 0.406],
        [0.229, 0.224, 0.225]
    )
])

testset = MyDataset(dir='/home/compu/Documents/exports/material_merge/test/',
                    image_ids='/home/compu/Documents/exports/newfile_material_merge.json',
                    transforms=transforms_test)

train_loader = DataLoader(trainset, batch_size=64, num_workers=8)
test_loader = DataLoader(testset, batch_size=64, num_workers=8)

print('학습 데이터셋 크기:', len(trainset))
print('테스트 데이터셋 크기:', len(testset))


학습 데이터셋 크기: 23197
테스트 데이터셋 크기: 5352


In [7]:
class EarlyStopping:
    """Early stops the training if loss doesn't improve after a given patience."""
    def __init__(self, patience=10, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when loss decrease.'''
        if self.verbose:
            self.trace_func(f'loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [8]:
# class MyModel(nn.Module):
#     def __init__(self) -> None:
#         super().__init__()
#         self.resnet = resnet50(pretrained=False)
#         self.classifier = nn.Linear(1000, 7)

#     def forward(self, x):
#         x = self.resnet(x)
#         x = self.classifier(x)

#         return x

model = resnet50(pretrained = True)
num_features = model.fc.in_features
model.fc = nn.Linear(num_features, 4)

model = model.to(device)
print(summary(model, input_size=(1, 3, 224, 224), verbose=0))

Layer (type:depth-idx)                   Output Shape              Param #
ResNet                                   [1, 4]                    --
├─Conv2d: 1-1                            [1, 64, 112, 112]         9,408
├─BatchNorm2d: 1-2                       [1, 64, 112, 112]         128
├─ReLU: 1-3                              [1, 64, 112, 112]         --
├─MaxPool2d: 1-4                         [1, 64, 56, 56]           --
├─Sequential: 1-5                        [1, 256, 56, 56]          --
│    └─Bottleneck: 2-1                   [1, 256, 56, 56]          --
│    │    └─Conv2d: 3-1                  [1, 64, 56, 56]           4,096
│    │    └─BatchNorm2d: 3-2             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-3                    [1, 64, 56, 56]           --
│    │    └─Conv2d: 3-4                  [1, 64, 56, 56]           36,864
│    │    └─BatchNorm2d: 3-5             [1, 64, 56, 56]           128
│    │    └─ReLU: 3-6                    [1, 64, 56, 56]           --
│ 

In [9]:
def focal_binary_cross_entropy(logits, targets, gamma=2):
    num_label = 4
    l = logits.reshape(-1)
    t = targets.reshape(-1)
    p = torch.sigmoid(l)
    p = torch.where(t >= 0.5, p, 1-p)
    logp = - torch.log(torch.clamp(p, 1e-4, 1-1e-4))
    loss = logp*((1-p)**gamma)
    loss = num_label*loss.mean()
    return loss

In [10]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = focal_binary_cross_entropy
wandb.init(project="kristin", name="resnet50_focal_aug2")
wandb.watch(model, criterion, log='all', log_freq=10)

num_epochs = 200
model.train()
start_time = time.time()

# early_stopping 객체 선언(5번의 epoch 연속으로 loss 미개선 시에 조기 종료 예시)
early_stopping = EarlyStopping(patience = 15, verbose = True, path = "material1.pt")

for epoch in range(num_epochs):
    for i, (images, targets) in enumerate(train_loader):
        optimizer.zero_grad()

        images = images.to(device)
        targets = targets.to(device)

        outputs = model(images)
        loss = criterion(outputs, targets)

        loss.backward()
        optimizer.step()

    outputs = (outputs > 0.5).float()
    acc = (outputs == targets).float().mean()
    metrics = {"train_loss": loss.item(), 
                "train_accuracy": acc.item()}
    wandb.log(metrics)

    early_stopping(loss.item(), model) # 수렴 earlystop 체크
    
    if early_stopping.early_stop: # 조건 만족 시 조기 종료
        break

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/home/compu/.local/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)
wandb: Currently logged in as: kew118. Use `wandb login --relogin` to force relogin


loss decreased (inf --> 0.354641).  Saving model ...
loss decreased (0.354641 --> 0.270631).  Saving model ...
loss decreased (0.270631 --> 0.237337).  Saving model ...
EarlyStopping counter: 1 out of 15
EarlyStopping counter: 2 out of 15
EarlyStopping counter: 3 out of 15
EarlyStopping counter: 4 out of 15
EarlyStopping counter: 5 out of 15
loss decreased (0.237337 --> 0.234349).  Saving model ...


In [ ]:
wandb.finish()

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    preds_list = []
    y_list = []
    
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)

        outputs = (outputs > 0.5).float()
        acc = (outputs == targets).float().mean()
        
        preds_list.append(outputs.float().cpu())
        y_list.append(targets.cpu())
    
    preds_list = np.concatenate(preds_list)
    y_list = np.concatenate(y_list)

    

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정확도(accuracy) 계산
accuracy = np.mean(y_list == preds_list)
print(f'Accuracy: {accuracy:.3f}')

# 정밀도(precision) 계산
precision = precision_score(y_list, preds_list, average='micro')
print(f'Precision: {precision:.3f}')

# 재현율(recall) 계산
recall = recall_score(y_list, preds_list, average='micro')
print(f'Recall: {recall:.3f}')

# F1 점수(F1 score) 계산
f1 = f1_score(y_list, preds_list, average='micro')
print(f'F1: {f1:.3f}')

In [ ]:
import sklearn.metrics as skm
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

print(skm.classification_report(y_list, preds_list))
matrix = multilabel_confusion_matrix(y_list, preds_list)
result = [mat.diagonal().sum() / mat.sum() for mat in matrix]
print(result)

In [ ]:
accuracy_score(y_list[:,0], preds_list[:,0])

저장된 모델 불러오기

In [ ]:
# 모델 불러오기 전 정의
class MyModel(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.resnet = resnet50(pretrained=True)
        self.classifier = nn.Linear(1000, 7)

    def forward(self, x):
        x = self.resnet(x)
        x = self.classifier(x)

        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = MyModel().to(device)
print(summary(model, input_size=(1, 3, 128, 128), verbose=0))

In [ ]:
# 모델 불러오기
model = MyModel()
model.load_state_dict(torch.load('material_end.pt'))

In [ ]:
model.eval()
start_time = time.time()

with torch.no_grad():
    preds_list = []
    y_list = []
    
    for i, (images, targets) in enumerate(test_loader):
        images = images.to(device)
        targets = targets.to(device)
        outputs = model(images)
        loss = criterion(outputs, targets)
        
        print(f'[예측 결과: {outputs[0]}] (실제 정답: {targets.data[0]})')
        
        if (i+1) % 10 == 0:
            outputs = outputs > 0.5
            acc = (outputs == targets).float().mean()
            preds_list.append(outputs.float())
            y_list.append(targets)
            print(f'{i+1}: {loss.item():.5f}, {acc.item():.5f}, {time.time() - start_time}')
    
    preds_list = np.concatenate(preds_list)
    y_list = np.concatenate(y_list)

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

# 정확도(accuracy) 계산
accuracy = np.mean(y_list == preds_list)
print(f'Accuracy: {accuracy:.3f}')

# 정밀도(precision) 계산
precision = precision_score(y_list, preds_list, average='micro')
print(f'Precision: {precision:.3f}')

# 재현율(recall) 계산
recall = recall_score(y_list, preds_list, average='micro')
print(f'Recall: {recall:.3f}')

# F1 점수(F1 score) 계산
f1 = f1_score(y_list, preds_list, average='micro')
print(f'F1: {f1:.3f}')